In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd drive/MyDrive/HW/661Project/

[Errno 2] No such file or directory: 'drive/MyDrive/HW/661Project/'
/content/drive/MyDrive/HW/661Project


In [26]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.bn2(self.conv2(out)))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [28]:
##Do Not Touch This Cell

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")

Train on GPU...


In [29]:
##Do Not Touch This Cell
max_epochs = 50

random_seed = 671
torch.manual_seed(random_seed)

In [30]:
data_transforms ={
    'train': transforms.Compose([
        transforms.RandomResizedCrop(32),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5],[.5, .5, .5])
    ]),
    'test': transforms.Compose([
        transforms.RandomResizedCrop(32),
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5],[.5, .5, .5])
    ])
}

train_set = torchvision.datasets.ImageFolder(root='./training set/', transform=data_transforms['train'])
train_loader = DataLoader(dataset=train_set, batch_size=128, shuffle=True, num_workers=2)

test_set = torchvision.datasets.ImageFolder(root='./testing set/', transform=data_transforms['test'])
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=True, num_workers=2)

In [31]:
loss_list, acc_list = [], []
loss_list_val, acc_list_val = [], []
criterion = nn.CrossEntropyLoss()

for epoch in range(max_epochs):
    #TODO: set the net to train mode:
    net.train()

    epoch_loss = 0.0
    correct = 0
    total_examples = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        ##TODO: pass the data into the network and store the output
        outputs = net(data)
        ##TODO: Calculate the cross entropy loss between the output and target 
        loss = criterion(outputs, labels)
        ##TODO: Perform backpropagation
        loss.backward()
        optimizer.step()

        ##TODO: Get the prediction from the output
        _, predicted = torch.max(outputs, 1)
        ##TODO: Calculate the correct number and add the number to correct
        correct += predicted.eq(labels).sum()
        total_examples += labels.size(0)
        ##TODO: Add the loss to epoch_loss.
        epoch_loss += loss.item()
    ##TODO: calculate the average loss
    avg_loss = epoch_loss / len(train_loader)
    ##TODO: calculate the average accuracy
    avg_acc = correct / total_examples
    ##TODO: append average epoch loss to loss list
    loss_list.append(avg_loss)
    ##TODO: append average accuracy to accuracy list
    acc_list.append(avg_acc)
        
    print('[epoch %d] loss: %.5f accuracy: %.4f' % (epoch + 1, avg_loss, avg_acc))

[epoch 1] loss: 1.15602 accuracy: 0.6138
[epoch 2] loss: 0.48928 accuracy: 0.7755
[epoch 3] loss: 0.33607 accuracy: 0.8753
[epoch 4] loss: 0.26707 accuracy: 0.8993
[epoch 5] loss: 0.24109 accuracy: 0.9005
[epoch 6] loss: 0.23055 accuracy: 0.9065
[epoch 7] loss: 0.20802 accuracy: 0.9185
[epoch 8] loss: 0.20532 accuracy: 0.9155
[epoch 9] loss: 0.17895 accuracy: 0.9303
[epoch 10] loss: 0.18084 accuracy: 0.9295
[epoch 11] loss: 0.17364 accuracy: 0.9318
[epoch 12] loss: 0.16669 accuracy: 0.9333
[epoch 13] loss: 0.15229 accuracy: 0.9418
[epoch 14] loss: 0.14441 accuracy: 0.9458
[epoch 15] loss: 0.15439 accuracy: 0.9420
[epoch 16] loss: 0.14096 accuracy: 0.9450
[epoch 17] loss: 0.14843 accuracy: 0.9423
[epoch 18] loss: 0.12924 accuracy: 0.9548
[epoch 19] loss: 0.12416 accuracy: 0.9593
[epoch 20] loss: 0.13040 accuracy: 0.9523
[epoch 21] loss: 0.11393 accuracy: 0.9560
[epoch 22] loss: 0.14063 accuracy: 0.9468
[epoch 23] loss: 0.11884 accuracy: 0.9560
[epoch 24] loss: 0.11176 accuracy: 0.9590
[

In [32]:
correct_test = 0
net.eval()
with torch.no_grad():
    for batch_idx, (data, label) in enumerate(test_loader):
        data, label = data.to(device), label.to(device)
        ##TODO: pass the data into the network and store the output
        outputs = net(data)
        ##TODO: Get the prediction from the output
        _, predicted = torch.max(outputs, 1)
        ##TODO: Calculate the correct number and add the number to correct_test
        correct_test += predicted.eq(label).sum()

print('Accuracy on the 10000 test images: %.2f %%' % (100 * correct_test / len(test_set)))

Accuracy on the 10000 test images: 87.65 %
